In [135]:
import re

import numpy as np
import pandas as pd
import spacy
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

nlp = spacy.load("es_core_news_lg")

In [136]:
file_path = './FAQs.txt'
file_handler = open(file_path, 'r', encoding='utf-8')
extracted_text = file_handler.read()

In [137]:
matches = re.findall(r'^-.*?(?=\n-|\Z)', extracted_text, re.MULTILINE | re.DOTALL)

In [138]:
question_answer_group = [re.sub(r'(\s)o(\s)', "\n", question, flags=re.UNICODE) for question in matches]
question_answer_group = [re.sub(r"[^A-Za-z \náéíóúñÑÁÉÍÓÚÑüÜ]", "", question, flags=re.UNICODE) for question in
                         question_answer_group]

In [139]:
lematized_questions = []
for question in question_answer_group:
    tokens = nlp(question)
    lematized_questions.append(" ".join([token.lemma_ for token in tokens if not token.is_stop]))

lematized_questions

['compatible equipo preinstalación ongrid \n   compatible necesitar comunicación instalación previo \n',
 'posibilidad conectar aerogenerador inversor híbrido \n GenPort salida aerogenerador AC señal estable \n',
 'conectar aerogenerador yó generador estar cliente conexión red \n GenPort aerogenerador generador Puerto Red Eléctricamente factible funcionamiento inestable \n',
 'tenéis compatibilidad inversor Huawei preinstalado \n   compatible comunicación \n',
 'tipo tensión emplear conexión paralelo embarrado \n   ocasión tensión bajo \n',
 'opción venta excedent energía red \n equipo funcionalidad venta excedent zero export to grid acumulación excedent batería \n',
 'Industrial Potencias Kw viable \n caso instalar SBR paralelo kw SBI Kw \n',
 'viable almacenamiento inferior Kwh \n instalar SBR paralelo Kwh SBI Kwh \n',
 'aumentar potencia Kw llegar Kw \n añadir kw adicional ongrid entrar Genport \n',
 'hacéis corriente alterno \n conectar paralelo alterna embarrado bajo tensión proye

In [140]:
df = pd.DataFrame(lematized_questions, columns=['questions'])
df['target'] = np.arange(len(lematized_questions))
df

,questions,target
0,compatible equipo preinstalación ongrid \n c...,0
1,posibilidad conectar aerogenerador inversor hí...,1
2,conectar aerogenerador yó generador estar clie...,2
3,tenéis compatibilidad inversor Huawei preinsta...,3
4,tipo tensión emplear conexión paralelo embarra...,4
...,...,...
142,solucionar conexión proyecto MT estudiar retro...,142
143,integrar sistema monitorización modelo retrofi...,143
144,gestionar SBI modelo retrofit repowering inter...,144
145,potencia nominal máximo configuración Sunbox ...,145


In [141]:
# Creamos un objeto CountVectorizer
vectorizer = CountVectorizer(stop_words=stopwords.words('spanish'))
corups_matrix = vectorizer.fit_transform(df['questions'])

In [142]:
naive_bayes = MultinomialNB()
_ = naive_bayes.fit(corups_matrix, df['target'])

In [143]:
search_input = "se resiente porque la energía de la red es mucha más cara "

search_clean = re.sub(r"[^A-Za-z\sáéíóúñÑÁÉÍÓÚÑüÜ]", "", search_input)

search_tokens = nlp(search_clean)
lemma_search = " ".join([token.lemma_ for token in search_tokens if not token.is_stop])
lemma_search

'resientir energía red cara'

In [144]:
search_vector = vectorizer.transform([lemma_search])

prediction = naive_bayes.predict(search_vector)

In [145]:
predictions_probability = naive_bayes.predict_proba(search_vector)
predictions_probability

array([[0.00405769, 0.00402003, 0.01178334, 0.00411501, 0.00405769,
        0.01571113, 0.00402003, 0.00403881, 0.00402003, 0.00402003,
        0.00405769, 0.00402003, 0.00403881, 0.00409579, 0.00413434,
        0.00415378, 0.00392778, 0.00415378, 0.00405769, 0.00402003,
        0.00411501, 0.00811538, 0.01167487, 0.00403881, 0.00796562,
        0.00382054, 0.00415378, 0.00413434, 0.00767632, 0.00409579,
        0.00396436, 0.00713607, 0.00815337, 0.00402003, 0.00411501,
        0.00394602, 0.00360047, 0.00394602, 0.00396436, 0.00392778,
        0.00402003, 0.00415378, 0.00407669, 0.00400137, 0.00403881,
        0.00371694, 0.0038737 , 0.00403881, 0.00348853, 0.00308199,
        0.01195901, 0.00670183, 0.00785556, 0.00332126, 0.00385588,
        0.00358421, 0.01514246, 0.00716842, 0.00323425, 0.00403881,
        0.00389162, 0.00405769, 0.00390965, 0.00407669, 0.00396436,
        0.00392778, 0.00398281, 0.00383816, 0.00800274, 0.00419302,
        0.00380303, 0.01075262, 0.00380303, 0.00

In [146]:
indices = np.argsort(predictions_probability[0])[-5:][::-1]
top_values = predictions_probability[0, indices]
top_columns = [matches[i] for i in indices]

print(f"Most accurate questions:")
for col, val in zip(top_columns, top_values):
    question = col.split("\n")[0]
    print(f'{question}: {val}')

Most accurate questions:
-	¿Un SBI puede trabajar sin planta FV ni producción FV?: 0.1381737974400379
-	Vuestro SMART MODE actual funciona con datos españoles. ¿Es posible decir cuándo y en qué condiciones el modo inteligente podría ser aplicable para otros países para diferenciarnos de otros proveedores?: 0.07175408145550516
-	¿Qué pasa en un proyecto de RETROFIT O REPOWERING con el parque FV existente si se cae la red? La energía FV del parque 1 puede aprovecharse o se corta por la característica del inversor ongrid?: 0.0434019461766863
-	Cuando la red se cae, ¿el cambia solo a modo aislada para alimentar la instalación dejando intactas las cargas críticas y empleando la energía FV de la planta existente?: 0.022713685555093482
-	¿Cómo se rentabilizan baterías, ya no son caras?: 0.02112037795723369
